In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as soup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import pandas as pd
import time
from datetime import datetime, timedelta
import time
from IPython.display import clear_output
from config import conn_host, conn_database, conn_user, conn_password
import mysql.connector
from dateutil import tz

In [2]:
months = dict(Jan=1,Feb=2,Mar=3,Apr=4,May=5,Jun=6,Jul=7,Aug=8,Sep=9,Oct=10,Nov=11,Dec=12)

# If you want to change the league, browse through the odds portal website to find the correct url parameters for you
country = "germany"
league = "bundesliga"

In [3]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
    except Exception as e:
        print(e)
    db.close()
    return resp

In [4]:
execute_query("ALTER TABLE matches " +
              "ADD home_odds FLOAT NULL," +
              "ADD away_odds FLOAT NULL," +
              "ADD draw_odds FLOAT NULL;")

Execution failed on sql 'ALTER TABLE matches ADD home_odds FLOAT NULL,ADD away_odds FLOAT NULL,ADD draw_odds FLOAT NULL;': 1060 (42S21): Duplicate column name 'home_odds'


C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
teams = execute_query("SELECT * FROM teams")

In [6]:
print(teams)

        id            name
0       33  Manchester Utd
1       34       Newcastle
2       35     Bournemouth
3       36          Fulham
4       37    Huddersfield
..     ...             ...
265  12277        Ipatinga
266  13173      Chivas USA
267  13974      Santa Cruz
268  16489       Austin FC
269  18310       Charlotte

[270 rows x 2 columns]


In [20]:
def get_betting_odds(season):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/Sao_Paulo')
    if '2014' in season and league == "j1-league": # Different name for J1 league for the 2014 season
        base_url = f"https://www.oddsportal.com/soccer/{country}/j-league{season}/results/"
    else:
        base_url = f"https://www.oddsportal.com/soccer/{country}/{league}{season}/results/"
    option = Options()
    option.headless = True
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=option)
    driver.get(base_url)
    time.sleep(5)
    
    try:
        button = driver.find_element("xpath", '/html/body/div[3]/div[2]/div/div[1]/div/div[2]/div/button[1]')
        button.click()
    except:
        print('A problem occoured while trying to close the cookies popup')
    button = driver.find_element("xpath", '/html/body/div[1]/div/div[2]/div[6]/div[1]/div/div[1]/div[2]/div[1]/div[6]/div/a[last()]')
    button.click()
    time.sleep(2)
    
    element = driver.find_element('id', "tournamentTable")
    htmlContent = element.get_attribute('outerHTML')
    page_soup = soup(htmlContent, "html.parser")
    active_page = int(''.join(page_soup.find('span', {"class": "active-page"}).findAll(text=True)))
    print('{}: Page {}'.format(season[1:], active_page))
    
    games = []
    
    while int(active_page) >= 1:
        dates = page_soup.findAll('tr', {"class": "center nob-border"})
        games_registered = 0
        
        for i in range(len(dates), 0, -1):
            try:
                if len(dates) == i:
                    date_games = dates[i-1].find_next_siblings('tr', {"class": "deactivate"})
                else:
                    date_games = dates[i-1].find_next_siblings('tr', {"class": "deactivate"})[:-1*games_registered]
                games_registered += len(date_games)
                date_info_splitted = dates[i-1].contents[0].text.split('-')
                if len(date_info_splitted) > 1 and 'stage' not in date_info_splitted[1]: # Include named stages that are not playoffs
                    continue
                date_text = date_info_splitted[0].strip()
                for g in date_games:
                    try:
                        unix_date = datetime(int(date_text.split(' ')[2]), 
                                                        months[date_text.split(' ')[1]], 
                                                        int(date_text.split(' ')[0]), 
                                                        hour=int(g.contents[0].text.strip().split(':')[0]), 
                                                        minute=int(g.contents[0].text.strip().split(':')[1]), 
                                                        second=0).replace(tzinfo=from_zone).astimezone(to_zone)
                        home_team_string = g.contents[1].text.split(' - ')[0].strip()
                        away_team_string = g.contents[1].text.split(' - ')[1].strip()
                        home_team = teams.loc[teams['name'].str.lower() == home_team_string.lower()].iloc[0]
                        away_team = teams.loc[teams['name'].str.lower() == away_team_string.lower()].iloc[0]
                        games_parsed = [unix_date,
                                          home_team['id'], # Home Team Id
                                          away_team['id'], # Away Team Id
                                          home_team['name'], # Team A Name
                                          away_team['name'], # Team B Name
                                          g.contents[2].text.split(':')[0].strip(), # Team A Score
                                          g.contents[2].text.split(':')[1].replace('OT', '').strip(), # Team B Score
                                          g.contents[3].text,  # Team A Odds
                                          g.contents[4].text,  # Draw Odds
                                          g.contents[5].text]  # Team B Odds
                        games.append(games_parsed)
                    except:
                        continue
                # print('{} Games appended'.format(len(games_parsed)))
            except Exception as e:
                print(e)
                continue
        
        btn_next_page = driver.find_element("xpath", '/html/body/div[1]/div/div[2]/div[6]/div[1]/div/div[1]/div[2]/div[1]/div[6]/div/a[2]')
        btn_next_page.click()
        time.sleep(4)
        
        element = driver.find_element('id', "tournamentTable")
        htmlContent = element.get_attribute('outerHTML')
        page_soup = soup(htmlContent, "html.parser")
        active_page -= 1
        clear_output(wait=True)
        print('{}: Page {}'.format(season[1:], active_page))
    
    driver.quit()
    return games
    

In [21]:
season = 2012
games = []
# while season < 2022:
# #     games.extend(get_betting_odds(f'-{season}')) # Brazil/Same year leagues
#     games.extend(get_betting_odds(f'-{season}-{season+1}')) # European/Two year leagues leagues

#     season += 1

games.extend(get_betting_odds('')) # For the current season

odds_df = pd.DataFrame(games, columns=['GAME_DATE', 'TEAM_A_ID', 'TEAM_B_ID', 'TEAM_A', 'TEAM_B', 'TEAM_A_SCORE', 'TEAM_B_SCORE', 'TEAM_A_ODDS', 'DRAW_ODDS', 'TEAM_B_ODDS'])
print(len(games))

: Page 0
135


In [22]:
odds_df.head()

,GAME_DATE,TEAM_A_ID,TEAM_B_ID,TEAM_A,TEAM_B,TEAM_A_SCORE,TEAM_B_SCORE,TEAM_A_ODDS,DRAW_ODDS,TEAM_B_ODDS
0,2022-08-05 15:30:00-03:00,169,157,Eintracht Frankfurt,Bayern Munich,1,6,5.95,5.11,1.49
1,2022-08-06 13:30:00-03:00,165,168,Dortmund,Bayer Leverkusen,1,0,2.05,4.03,3.30
2,2022-08-06 10:30:00-03:00,170,160,Augsburg,Freiburg,0,4,3.49,3.55,2.14
3,2022-08-06 10:30:00-03:00,163,167,B. Monchengladbach,Hoffenheim,3,1,2.06,3.98,3.34
4,2022-08-06 10:30:00-03:00,176,164,Bochum,Mainz,1,2,3.33,3.44,2.24


In [23]:
game_errors = []
for index, row in odds_df.iterrows():
    try:
        date_minus_one_day = row['GAME_DATE'] - timedelta(days=1)
        date_plus_one_day = row['GAME_DATE'] + timedelta(days=1)
        game_id = execute_query(f"SELECT id from matches WHERE (DATE_FORMAT(date,'%d/%m/%Y') = DATE_FORMAT('{row['GAME_DATE']}','%d/%m/%Y') OR DATE_FORMAT(date,'%d/%m/%Y') = DATE_FORMAT('{date_minus_one_day}','%d/%m/%Y') OR DATE_FORMAT(date,'%d/%m/%Y') = DATE_FORMAT('{date_plus_one_day}','%d/%m/%Y')) AND home_id = {row['TEAM_A_ID']} AND away_id = {row['TEAM_B_ID']}").iloc[0]['id']
        update_query = f"UPDATE matches SET home_odds = {row['TEAM_A_ODDS']}, away_odds = {row['TEAM_B_ODDS']}, draw_odds = {row['DRAW_ODDS']} WHERE id = {game_id}"
        execute_query(update_query, False)
    except Exception as e:
        new_item = dict()
        new_item['error'] = e
        new_item['match'] = row
        game_errors.append(new_item)
    clear_output(wait=True)
    print(f"{index}/{len(odds_df.index)}")

134/135


In [24]:
if len(game_errors) == 0:
    print('There were no errors while reading games from the Odds Portal website.')
else:
    print('The following games had an error while trying to read games from the Odds Portal website:')
    for item in game_errors:
        print(f"\n{item['match']['GAME_DATE']}: {item['match']['TEAM_A']}({item['match']['TEAM_A_ID']}) {item['match']['TEAM_A_SCORE']} x {item['match']['TEAM_B_SCORE']} {item['match']['TEAM_B']}({item['match']['TEAM_B_ID']})")
        print(f"ERROR: {item['error']}")

There were no errors while reading games from the Odds Portal website.


In [25]:
fixtures_df = execute_query(f"SELECT m.id, m.date, m.season, l.id AS league_id, l.name AS league, ht.name as home_team, at.name as away_team, m.home_score, m.away_score, m.home_odds, m.away_odds, m.draw_odds FROM matches AS m INNER JOIN teams AS ht ON (m.home_id = ht.id) INNER JOIN teams AS at ON (m.away_id = at.id) INNER JOIN leagues AS l ON (m.league_id = l.id) WHERE m.season = 2022 and league_id = 253 ORDER BY m.date ASC")

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [26]:
fixtures_df.tail()

,id,date,season,league_id,league,home_team,away_team,home_score,away_score,home_odds,away_odds,draw_odds
456,817174,2022-10-01 23:30:00,2022,253,Major League Soccer (USA),Los Angeles Galaxy,Real Salt Lake,1,1,1.67,4.63,4.23
457,817169,2022-10-02 14:00:00,2022,253,Major League Soccer (USA),New York City,Orlando City,2,1,1.87,4.22,3.55
458,817175,2022-10-02 16:00:00,2022,253,Major League Soccer (USA),Portland Timbers,Los Angeles FC,1,2,3.10,2.14,3.79
459,817176,2022-10-02 18:00:00,2022,253,Major League Soccer (USA),Sporting Kansas City,Seattle Sounders,1,0,2.19,3.13,3.60
460,817177,2022-10-02 21:30:00,2022,253,Major League Soccer (USA),Nashville SC,Houston Dynamo,1,2,NaN,NaN,NaN
